In [11]:
from trino.dbapi import Connection, connect

conn: Connection = connect(
    host="87.242.86.99",
    port="8081",
    user="admin",
    catalog="iceberg",
    schema="default",
)
cur = conn.cursor()
cur.execute("SELECT * FROM system.runtime.nodes")
rows = cur.fetchall()
rows

[['0785fae03b1f', 'http://192.168.112.8:8080', '447', True, 'active']]

In [12]:
# list catalogs
rows = cur.execute("SHOW CATALOGS")
rows.fetchall()

[['iceberg'], ['jmx'], ['memory'], ['system'], ['tpcds'], ['tpch']]

In [13]:
cur.execute("CREATE SCHEMA IF NOT EXISTS iceberg.default WITH (location = 's3://warehouse/')")

In [14]:
cur.execute(
    """CREATE TABLE IF NOT EXISTS example_table (c1 INTEGER, c2 DATE, c3 DOUBLE) WITH (format = 'PARQUET', partitioning = ARRAY['c1', 'c2'], sorted_by = ARRAY['c3'], location = 's3://warehouse/default/example_table/')"""
)

In [15]:
cur.execute("SHOW TABLES")
rows = cur.fetchall()
rows

[['example_table'], ['groups']]

In [16]:
cur.execute("DESCRIBE iceberg.default.example_table")
cur.fetchall()

[['c1', 'integer', '', ''], ['c2', 'date', '', ''], ['c3', 'double', '', '']]

In [18]:
cur.execute("INSERT INTO iceberg.default.example_table VALUES (1, DATE '2022-01-01', 4.0)")

In [49]:
import pandas as pd
cur.execute('SELECT * FROM "example_table$snapshots"')
table = pd.DataFrame(cur.fetchall(), columns=[i[0] for i in cur.description])
table

,committed_at,snapshot_id,parent_id,operation,manifest_list,summary
0,2024-05-10 16:23:59.153000+03:00,340838695005770323,NaN,append,s3://warehouse/default/example_table/metadata/...,"{'changed-partition-count': '0', 'total-equali..."
1,2024-05-10 16:26:01.470000+03:00,5226348749357996666,3.408387e+17,append,s3://warehouse/default/example_table/metadata/...,"{'changed-partition-count': '1', 'added-data-f..."
2,2024-05-10 19:32:10.705000+03:00,1853476678964347994,5.226349e+18,append,s3://warehouse/default/example_table/metadata/...,"{'changed-partition-count': '1', 'added-data-f..."
3,2024-05-10 19:32:26.227000+03:00,3185844970411892490,1.853477e+18,append,s3://warehouse/default/example_table/metadata/...,"{'changed-partition-count': '1', 'added-data-f..."


In [52]:
cur.execute("CALL iceberg.system.rollback_to_snapshot('default', 'example_table', 3185844970411892490)")

In [51]:
cur.execute('SELECT * from "example_table$history"')
pd.DataFrame(cur.fetchall())

,0,1,2,3
0,2024-05-10 16:23:59.153000+03:00,340838695005770323,NaN,True
1,2024-05-10 16:26:01.470000+03:00,5226348749357996666,3.408387e+17,True
2,2024-05-10 19:32:10.705000+03:00,1853476678964347994,5.226349e+18,False
3,2024-05-10 19:32:26.227000+03:00,3185844970411892490,1.853477e+18,False


In [58]:
cur.execute('SELECT * FROM "example_table" FOR VERSION AS OF 1853476678964347994')
rows = cur.fetchall()
rows

[[1, datetime.date(2021, 1, 1), 1.0], [1, datetime.date(2022, 1, 1), 3.0]]

In [58]:
cur.execute("SHOW COLUMNS FROM iceberg.default.groups")
cur.fetchall()

[['название', 'varchar', '', ''], ['специальность', 'varchar', '', '']]

In [60]:
cur.execute("SELECT * FROM iceberg.nyc.taxis")
cur.fetchall()

[[1,
  datetime.datetime(2021, 4, 1, 0, 0, 18),
  datetime.datetime(2021, 4, 1, 0, 21, 54),
  1.0,
  8.4,
  1.0,
  'N',
  79,
  116,
  1,
  25.5,
  3.0,
  0.5,
  5.85,
  0.0,
  0.3,
  35.15,
  2.5,
  0.0],
 [1,
  datetime.datetime(2021, 4, 1, 0, 42, 37),
  datetime.datetime(2021, 4, 1, 0, 46, 23),
  1.0,
  0.9,
  1.0,
  'N',
  75,
  236,
  2,
  5.0,
  3.0,
  0.5,
  0.0,
  0.0,
  0.3,
  8.8,
  2.5,
  0.0],
 [1,
  datetime.datetime(2021, 4, 1, 0, 57, 56),
  datetime.datetime(2021, 4, 1, 1, 8, 22),
  1.0,
  3.4,
  1.0,
  'N',
  236,
  168,
  2,
  11.5,
  3.0,
  0.5,
  0.0,
  0.0,
  0.3,
  15.3,
  2.5,
  0.0],
 [1,
  datetime.datetime(2021, 4, 1, 0, 1, 58),
  datetime.datetime(2021, 4, 1, 0, 54, 27),
  1.0,
  0.0,
  1.0,
  'N',
  47,
  61,
  1,
  44.2,
  0.0,
  0.5,
  0.0,
  0.0,
  0.3,
  45.0,
  0.0,
  0.0],
 [2,
  datetime.datetime(2021, 4, 1, 0, 24, 55),
  datetime.datetime(2021, 4, 1, 0, 34, 33),
  1.0,
  1.96,
  1.0,
  'N',
  238,
  152,
  1,
  9.0,
  0.5,
  0.5,
  3.09,
  0.0,
  0.3,

In [88]:
# select from iceberg.default.groups
cur.execute("SELECT * FROM iceberg.nyc.taxis LIMIT 10")
rows = cur.fetchall()
rows

[[1,
  datetime.datetime(2021, 4, 1, 0, 0, 18),
  datetime.datetime(2021, 4, 1, 0, 21, 54),
  1.0,
  8.4,
  1.0,
  'N',
  79,
  116,
  1,
  25.5,
  3.0,
  0.5,
  5.85,
  0.0,
  0.3,
  35.15,
  2.5,
  0.0],
 [1,
  datetime.datetime(2021, 4, 1, 0, 42, 37),
  datetime.datetime(2021, 4, 1, 0, 46, 23),
  1.0,
  0.9,
  1.0,
  'N',
  75,
  236,
  2,
  5.0,
  3.0,
  0.5,
  0.0,
  0.0,
  0.3,
  8.8,
  2.5,
  0.0],
 [1,
  datetime.datetime(2021, 4, 1, 0, 57, 56),
  datetime.datetime(2021, 4, 1, 1, 8, 22),
  1.0,
  3.4,
  1.0,
  'N',
  236,
  168,
  2,
  11.5,
  3.0,
  0.5,
  0.0,
  0.0,
  0.3,
  15.3,
  2.5,
  0.0],
 [1,
  datetime.datetime(2021, 4, 1, 0, 1, 58),
  datetime.datetime(2021, 4, 1, 0, 54, 27),
  1.0,
  0.0,
  1.0,
  'N',
  47,
  61,
  1,
  44.2,
  0.0,
  0.5,
  0.0,
  0.0,
  0.3,
  45.0,
  0.0,
  0.0],
 [2,
  datetime.datetime(2021, 4, 1, 0, 24, 55),
  datetime.datetime(2021, 4, 1, 0, 34, 33),
  1.0,
  1.96,
  1.0,
  'N',
  238,
  152,
  1,
  9.0,
  0.5,
  0.5,
  3.09,
  0.0,
  0.3,